In [1]:
class Assumable(object):
    """An askable atom"""

    def __init__(self, atom):
        """clause with atom head and lost of atoms body"""
        self.atom = atom

    def __str__(self):
        """returns the string representation of a clause.
        """
        return 'assumable ' + self.atom + '.'


In [2]:
from lib.logic_problem import Clause, Askable, KB, yes


class KBA(KB):
    """A knowledge base that can include assumables"""
    
    def __init__(self, statements):
        self.assumables = [c.atom for c in statements if isinstance(c, Assumable)]
        KB.__init__(self, statements)
        
    def prove_all_ass(self, ans_body, assumed=set()):
        """returns a list of sets of assumables that extends assumed 
        to imply ans_body from self.
        ans_body is a list of atoms (it is the body of the answer clause).
        assumed is a set of assumables already assumed
        """
        if ans_body:
            selected = ans_body[0]   # select first atom from ans_body
            if selected in self.askables:
                if yes(input('Is ' + selected + ' true? ')):
                    return self.prove_all_ass(ans_body[1:], assumed)
                else:
                    return []   # no answers
            elif selected in self.assumables:
                return self.prove_all_ass(ans_body[1:], assumed | {selected})
            else:
                return [ass
                        for cl in self.clauses_for_atom(selected)
                        for ass in self.prove_all_ass(cl.body + ans_body[1:], assumed)
                       ]  # union of answers for each clause with head=selected
        else:                 # empty body
            return [assumed]    # one answer

    def conflicts(self):
        """returns a list of minimal conflicts"""
        return minsets(self.prove_all_ass(['false']))


In [3]:
def minsets(ls):
    """ls is a list of sets
    returns a list of minimal sets in ls
    """  
    ans = []     # elements known to be minimal
    for c in ls:
        if not any(c1 < c for c1 in ls) and not any(c1 <= c for c1 in ans):
            ans.append(c)
    return ans


# minsets([{2, 3, 4}, {2, 3}, {6, 2, 3}, {2, 3}, {2, 4, 5}])
def diagnoses(cons):
    """cons is a list of (minimal) conflicts.
    returns a list of diagnoses."""
    if cons == []:
        return [set()]
    else:
        return minsets([(
                           {e} | d)                # | is set union
                           for e in cons[0]
                           for d in diagnoses(cons[1:])
                       ])


In [5]:
electa = KBA([
    Clause('light_l1'),
    Clause('light_l2'),
    Assumable('ok_l1'),
    Assumable('ok_l2'),
    Assumable('ok_s1'),
    Assumable('ok_s2'),
    Assumable('ok_s3'),
    Assumable('ok_cb1'),
    Assumable('ok_cb2'),
    Assumable('live_outside'),
    Clause('live_l1', ['live_w0']),
    Clause('live_w0', ['up_s2','ok_s2','live_w1']),
    Clause('live_w0', ['down_s2','ok_s2','live_w2']),
    Clause('live_w1', ['up_s1', 'ok_s1', 'live_w3']),
    Clause('live_w2', ['down_s1', 'ok_s1','live_w3' ]),
    Clause('live_l2', ['live_w4']),
    Clause('live_w4', ['up_s3','ok_s3','live_w3' ]),
    Clause('live_p_1', ['live_w3']),
    Clause('live_w3', ['live_w5', 'ok_cb1']),
    Clause('live_p_2', ['live_w6']),
    Clause('live_w6', ['live_w5', 'ok_cb2']),
    Clause('live_w5', ['live_outside']),
    Clause('lit_l1', ['light_l1', 'live_l1', 'ok_l1']),
    Clause('lit_l2', ['light_l2', 'live_l2', 'ok_l2']),
    Askable('up_s1'),
    Askable('down_s1'),
    Askable('up_s2'),
    Askable('down_s2'),
    Askable('up_s3'),
    Askable('down_s2'),
    Askable('dark_l1'),
    Askable('dark_l2'),
    Clause('false', ['dark_l1', 'lit_l1']),
    Clause('false', ['dark_l2', 'lit_l2'])
])

electa.prove_all_ass(['false'])
cs=electa.conflicts()
print(cs)
diagnoses(cs)        # diagnoses from conflicts


Is dark_l1 true? y
Is up_s2 true? y
Is up_s1 true? y
Is down_s2 true? y
Is down_s1 true? n
Is dark_l2 true? n
Is dark_l1 true? y
Is up_s2 true? n
Is down_s2 true? y
Is down_s1 true? y
Is dark_l2 true? y
Is up_s3 true? n
[{'ok_cb1', 'ok_s1', 'ok_l1', 'live_outside', 'ok_s2'}]


[{'ok_cb1'}, {'ok_s1'}, {'ok_l1'}, {'live_outside'}, {'ok_s2'}]